## Process

- Each model has it's own csv file from which it queries the next points in the AL loop
- This csv links back to enumerated grid/pool of experiments (csv with volumes and concentrations)
- Separate dataframe for model training that has the same indices 
- Experimentalists need volume data
- Models do not need volume data
https://modal-python.readthedocs.io/en/latest/content/examples/pool-based_sampling.html

In [1]:
import pandas as pd
import qgrid

In [2]:
df = pd.read_csv('knn.csv')
training_raw = pd.read_csv('../data/raw/0057.perovskitedata_DRPFeatures_2020-07-02.csv')

X_train = training_raw[['_rxn_M_acid', '_rxn_M_inorganic', '_rxn_M_organic']]
Y_train = training_raw[['_out_crystalscore']]
stateset_raw = pd.read_csv('../data/phase3/Me2NH2I/Me2NH2I_10uL_stateset.csv')
X_stateset = stateset_raw[['_rxn_M_acid', '_rxn_M_inorganic', '_rxn_M_organic']]
vol_stateset = stateset_raw[['Reagent1 (ul)','Reagent2 (ul)','Reagent3 (ul)','Reagent7 (ul)']]
X_stateset

,_rxn_M_acid,_rxn_M_inorganic,_rxn_M_organic
0,14.842494,0.0000,1.012
1,14.842494,0.1860,0.966
2,14.842494,0.1488,0.966
3,14.842494,0.1116,0.966
4,14.842494,0.0744,0.966
...,...,...,...
20353,0.000000,0.0372,0.092
20354,0.000000,0.0000,0.092
20355,0.000000,0.0372,0.046
20356,0.000000,0.0000,0.046


In [3]:
widget = qgrid.show_grid(df, show_toolbar=True)
widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from modAL.models import ActiveLearner

knn = KNeighborsClassifier(n_neighbors=1)
learner = ActiveLearner(estimator=knn, X_training=X_train.to_numpy(), y_training=Y_train.to_numpy().ravel())

In [28]:
query_index, query_instance = learner.query(X_stateset.to_numpy())
print(query_index, query_instance)
vol_stateset.iloc[query_index]

[0] [[14.84249403  0.          1.012     ]]


,Reagent1 (ul),Reagent2 (ul),Reagent3 (ul),Reagent7 (ul)
0,0,0,220,280


In [30]:
score = [1]
learner.teach(query_instance, score)